In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [21]:
import pandas as pd
import numpy as np

import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

DIAGS_CONGENITOS = [
    "Q200",
    "Q201",
    "Q202",
    "Q203",
    "Q204",
    "Q205",
    "Q206",
    "Q208",
    "Q209",
    "Q210",
    "Q211",
    "Q212",
    "Q213",
    "Q214",
    "Q218",
    "Q219",
    "Q220",
    "Q221",
    "Q222",
    "Q223",
    "Q224",
    "Q225",
    "Q226",
    "Q228",
    "Q229",
    "Q230",
    "Q231",
    "Q232",
    "Q233",
    "Q234",
    "Q238",
    "Q239",
    "Q240",
    "Q241",
    "Q242",
    "Q243",
    "Q244",
    "Q245",
    "Q246",
    "Q248",
    "Q249",
    "Q250",
    "Q251",
    "Q252",
    "Q253",
    "Q254",
    "Q255",
    "Q256",
    "Q257",
    "Q258",
    "Q259",
    "Q260",
    "Q261",
    "Q262",
    "Q263",
    "Q264",
    "Q265",
    "Q268",
    "Q269",
    "Q288",
    "Q289",
    "Q336",
    "Q339",
]

In [3]:
# Lee la base de datos
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
    dtypes={"EDAD_CALCULADA_ANO": pl.Float64}
)

In [7]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "PERTENENCIA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]
metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

In [13]:
metricas = metricas.sort(by=["ANO_EGRESO", "DIAG1", "n_egresos"], descending=True)

In [24]:
metricas = metricas.with_columns(
    pl.col("n_egresos")
    .rank("min", descending=True)
    .over(["ANO_EGRESO", "DIAG1"])
    .alias("ranking_egresos")
)

metricas_congenitos = metricas.filter(pl.col("DIAG1").is_in(DIAGS_CONGENITOS))

In [13]:
# NeoNato es hasta 1 mes (sin incluirlo)
df_neonato = df_nacional.filter(pl.col("EDAD_CALCULADA_ANO") < 0.083333)

# Ninos es de 1 mes hasta 15 anios
df_ninos = df_nacional.filter(
    (pl.col("EDAD_CALCULADA_ANO") >= 0.083333) & (pl.col("EDAD_CALCULADA_ANO") < 15)
)

# Adolescente es de 15 anios hasta 18
df_adolescente = df_nacional.filter(
    (pl.col("EDAD_CALCULADA_ANO") >= 15) & (pl.col("EDAD_CALCULADA_ANO") < 18)
)

# El total del RDR seria desde los 0 anios hasta los 18
df_hasta_18 = df_nacional.filter((pl.col("EDAD_CALCULADA_ANO") < 18))

In [14]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "PERTENENCIA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

# Define los grupos que se quieren exportar
rankings_a_exportar = {
    "neonato": df_neonato,
    "ninos": df_ninos,
    "adolescente": df_adolescente,
    "hasta_18": df_hasta_18,
}

In [15]:
# Itera en los rangos etarios para exportar los rankings
for rango_etario, df_etario in rankings_a_exportar.items():
    print(rango_etario)
    metricas = build_features.obtener_metricas_egresos(df_etario, AGRUPACION).collect()
    metricas.write_csv(f"../data/interim/resumen_egresos_nacionales_{rango_etario}.csv")

neonato
ninos
adolescente
hasta_18
